task name: Resolve Expression Task
task description: Implement the provided C# code as a dotnet interactive notebook

In [ ]:
#r "nuget:Microsoft.ML"


In [ ]:
using System;
using System.Collections.Generic;
using Microsoft.ML;
using Microsoft.ML.Data;

In [ ]:
public class InputData
{
    public float FloatColumn;
    [VectorType(3)]
    public float[] FloatVectorColumn;
    public int IntColumn;
    public string StringColumn;
    public bool BooleanColumn;
    [VectorType(2)]
    public string[] StringVectorColumn;

    public InputData(float f, float[] fv, int i, string s, bool b, string[] sv)
    {
        FloatColumn = f;
        FloatVectorColumn = fv;
        IntColumn = i;
        StringColumn = s;
        BooleanColumn = b;
        StringVectorColumn = sv;
    }
}

var mlContext = new MLContext();

var samples = new List<InputData>()
{
    new InputData(0.5f, new[] { 1f, 0.2f }, 3, "hi", true, new[] { "zero", "one" }),
    new InputData(-2.7f, new[] { 3.5f, -0.1f }, 2, "bye", false, new[] { "a", "b" }),
    new InputData(1.3f, new[] { 1.9f, 3.3f }, 39, "hi", false, new[] { "0", "1" }),
    new InputData(3, new[] { 3f, 3f }, 4, "hello", true, new[] { "c", "d" }),
    new InputData(0, new[] { 1f, 1f }, 1, "hi", true, new[] { "zero", "one" }),
    new InputData(30.4f, new[] { 10f, 4f }, 9, "bye", true, new[] { "e", "f" }),
    new InputData(5.6f, new[] { 1.1f, 2.2f }, 0, "hey", false, new[] { "g", "h" }),
};

var dataview = mlContext.Data.LoadFromEnumerable(samples);

In [ ]:
var pipeline = mlContext.Transforms.Expression("Expr1", "(x,y)=>log(y)+x",
        nameof(InputData.FloatColumn), nameof(InputData.FloatVectorColumn))
    .Append(mlContext.Transforms.Expression("Expr2", "(b,s,i)=>b ? len(s) : i",
        nameof(InputData.BooleanColumn), nameof(InputData.StringVectorColumn), nameof(InputData.IntColumn)))
    .Append(mlContext.Transforms.Expression("Expr3", "(s,f1,f2,i)=>len(concat(s,\"a\"))+f1+f2+i",
        nameof(InputData.StringColumn), nameof(InputData.FloatVectorColumn), nameof(InputData.FloatColumn), nameof(InputData.IntColumn)))
    .Append(mlContext.Transforms.Expression("Expr4", "(x,y)=>cos(x+pi())*y",
        nameof(InputData.FloatColumn), nameof(InputData.IntColumn)));

In [ ]:
var transformedData = pipeline.Fit(dataview).Transform(dataview);

In [ ]:
public sealed class TransformedData
{
    public float[] Expr1 { get; set; }
    public int[] Expr2 { get; set; }
    public float[] Expr3 { get; set; }
    public double Expr4 { get; set; }
}

var featuresColumn = mlContext.Data.CreateEnumerable<TransformedData>(
    transformedData, reuseRowObject: false);

Console.WriteLine($"Features column obtained post-transformation.");
foreach (var featureRow in featuresColumn)
{
    Console.Write(string.Join(" ", featureRow.Expr1));
    Console.Write(" ");
    Console.Write(string.Join(" ", featureRow.Expr2));
    Console.Write(" ");
    Console.Write(string.Join(" ", featureRow.Expr3));
    Console.Write(" ");
    Console.WriteLine(featureRow.Expr4);
}